# Goals
1. Download ONNX model + config from HL
2. Get relevant parameters (e.g. input size, class list) out of config or model file
3. ~Download image from HL~
4. ~Instantiate ONNX model~
4. ~Perform inference on image~
5. ~Display results (save to file)~

# House Keeping

In [ ]:
def i_am_running_in_colab():
    try:
        import google.colab
        return True
    except:
        return False
    
if i_am_running_in_colab():
    %env PYPI_USERNAME="..."
    %env PYPI_PASSWORD="..."
    !git clone https://github.com/tall-josh/highlighter-client-v2-notebooks.git
    !bash highlighter-client-v2-notebooks/colab-scripts/setup-train-mmdet.sh
    !mkdir data

In [ ]:
# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

# Create a HLClient object from credentials

This client will be used when we need to communicate with Highlighter via GraphQL.

In [ ]:
HL_WEB_GRAPHQL_API_TOKEN="..."
HL_WEB_GRAPHQL_ENDPOINT="https://<account-name>.highlighter.ai/graphql"
img_id = 5568090
data_dir='data'

from highlighter_client.gql_client import HLClient
from highlighter_client.io import multithread_graphql_image_download

# Needed when using HighlighterClient in a notebook environment
HLClient._async = 'AUTO'
client = HLClient.from_credential(api_token=HL_WEB_GRAPHQL_API_TOKEN,
                                  endpoint_url=HL_WEB_GRAPHQL_ENDPOINT)

# download image for inference
multithread_graphql_image_download(client, 
                                   image_ids=[img_id], 
                                   image_dir=data_dir)

# Load ONNX model and perform inference

In [ ]:
onnx_model_file = f'{data_dir}/insulator_tie_model_epoch_15.onnx'
config_file = f'{data_dir}/fixed-model-config.py'
img_file = f'{data_dir}/{str(img_id)}.jpg'
output_inferences_file = f'{data_dir}/{str(img_id)}_with_inferences.jpg'

In [ ]:
import numpy as np
import onnx
from mmcv import Config
from mmdet.core.export import preprocess_example_input
from mmdet.core.export.model_wrappers import ONNXRuntimeDetector

In [ ]:
onnx_config = Config.fromfile(config_file)

# mystery: where do the 800 and 1216 come from?
# they're not the image scale in the config - that's (2400, 3200)
input_config = {'input_shape': (1,3,
                                800, # onnx_config.scales[0]['image_size'][0], 
                                1216 ), #onnx_config.scales[0]['image_size'][1]),
                'input_path': img_file,
                'normalize_cfg':onnx_config.img_norm_cfg,}

one_img, one_meta = preprocess_example_input(input_config)
img_list, img_meta_list = [one_img], [[one_meta]]
img_list = [_.cuda().contiguous() for _ in img_list]

In [ ]:
# Mystery: the first time you run this cell it appears to hang. You have to
# interrupt the kernel and rerun it, at which point it will finish fast
onnx_model = ONNXRuntimeDetector(onnx_model_file, 
                                 class_names=np.array(['Insulator', 'Tie Wire']), 
                                 device_id=0)

onnx_results = onnx_model(img_list, img_metas=img_meta_list, return_loss=False)[0]

In [ ]:
show_img = one_meta['show_img']
score_thr=0.5
onnx_model.show_result(
            show_img,
            onnx_results,
            score_thr=score_thr,
            show=True,
            win_name='ONNXRuntime',
            out_file=output_inferences_file)